# <center> TEXT TO SPEECH </center>

Notebook de test et d'implémentation de modèles de TTS.

## Modèles

On va premièrement utiliser le modèle [gTTS](https://gtts.readthedocs.io/en/latest/) qui est un modèle de TTS (Text To Speech) basé sur Google Translate. Celui-ci est très simple à utiliser, mais ne permet pas de modifier beaucoup de paramètres du modèle.

Modèles testés:
- [gTTS](https://gtts.readthedocs.io/en/latest/)
- [Tacotron2](https://pytorch.org/hub/nvidia_deeplearningexamples_tacotron2/)

Autres modèles de TTS :
- [Tacotron2](https://huggingface.co/transformers/model_doc/tacotron2.html) (exemple avec vocodeur HiFiGAN, sur modèle pré-entrainé avec le dataset LJSpeech [ici](https://huggingface.co/speechbrain/tts-tacotron2-ljspeech))
- [FastSpeech2](https://fastspeech2.github.io/fastspeech2/)

Vocodeurs:
- [HiFiGAN](https://pytorch.org/hub/nvidia_deeplearningexamples_hifigan/)
- [MelGAN](https://paperswithcode.com/method/melgan)
- WaveGlow
- WaveRNN

Modèle utilisé en pipeline : [gTTS](https://gtts.readthedocs.io/en/latest/)

Le modèle TacoTron2 est un modèle de TTS basé sur un encodeur-décodeur. Il prend en entrée un texte et génère un spectrogramme. Ce spectrogramme est ensuite passé dans un vocodeur qui génère le fichier audio. Grâce au spectrogramme généré, on peut espérer pouvoir évaluer la qualité du modèle en comparant les spectrogrammes de nos audio d'évaluation et ceux générés par le vocodeur.

## Evaluation

Il est très difficile d'évaluer la qualité d'un modèle de TTS. En effet, il n'existe pas de métrique permettant de quantifier la qualité d'un fichier audio. On peut cependant utiliser des métriques de similarité entre spectrogrammes, ou encore des métriques de similarité entre les fichiers audios. Cependant, ces métriques ne sont pas forcément représentatives de la qualité du fichier audio.

Dans notre cas nous allons principalement nous reposer sur le MOS (Mean Opinion Score) qui est une métrique subjective. Celle-ci est obtenue en faisant écouter un fichier audio à un panel de personnes, et en leur demandant de noter la qualité du fichier audio entre 1 et 5. On peut ensuite calculer la moyenne des notes obtenues pour obtenir le MOS.

Nous allons utiliser le dataset [LibriSpeech](https://huggingface.co/datasets/librispeech_asr) qui est un dataset de fichiers audios de livres audio. Ce dataset est composé de 1000h d'audio environ. Chaque fichier audio est accompagné d'un transcript. Nous allons utiliser ce transcript pour générer un fichier audio avec notre modèle de TTS, et ensuite comparer le fichier audio généré avec le fichier audio original.

Comme dit précédemment, nous allons aussi essayer d'évaluer les audio en comparant les spectrogrammes (mel cepstral distortion). Cela peut se faire en utilisant Tacotron2 qui génère un spectrogramme à partir d'un texte de jfleg, et en comparant ce spectrogramme avec celui récupéré de l'audio jgelf correspondant au texte.

### Import des librairies

In [2]:
!pip3 install -q -U deep_phonemizer

import torchaudio, torch, os
import IPython.display as ipd
from torch.utils.data import Dataset, DataLoader

from gtts import gTTS
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

## Définition des fonctions

Nous allons définir les fonctions permettant de charger les datasets et de les traiter.

In [3]:
class GEC_Dataset(Dataset):
    def __init__(self, path, text):
        """
        Dataset pour les données en sortie du GEC
        :param path: Chemin des données
        :param text: Texte corrigé (Default None, on utilise les données du path. Pour une utilisation du TTS en pipeline alors text est le texte corrigé)
        """
        self.sentences = []
        
        if text is not None:
            self.sentences.append(text.strip())
        else:
            self.path = path
            self.files = os.listdir(path)
            for file in self.files:
                with open(path + '\\' + file, 'r', encoding='utf-8') as file:
                    for line in file:
                        self.sentences.append(line.strip())
    
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        transcript = self.sentences[idx]

        return transcript, idx


def load_librispeech_dataset(path='data\\'):
    """
    Fonction permettant de charger le dataset LibriSpeech
    :param path: Chemin du dataset
    :return: Dataloader du dataset
    """
    
    # LibriSpeech
    librispeech_dataset = torchaudio.datasets.LIBRISPEECH(path, url='dev-clean', download=True)
    # Réduction du dataset pour les tests (500 samples)
    librispeech_dataset_short = torch.utils.data.Subset(librispeech_dataset, range(500))
    librispeech_dataloader = DataLoader(librispeech_dataset_short, batch_size=1)
    
    print("Number of samples:", len(librispeech_dataset_short))
    
    sample_meta0 = librispeech_dataset.get_metadata(0)
    sample_0 = librispeech_dataset_short[0][0]
    print("""
    Audio n°0 :
    
    Path audio: {}{}
    Sample rate : {}
    Transcript : {}
    Speaker ID : {}
    Chapter ID : {}
    Utterance ID : {}
    """.format(path + "LibriSpeech", *sample_meta0))
    
    #Playing sample_O
    ipd.Audio(sample_0, rate=sample_meta0[1])
    
    return librispeech_dataloader

def process_librispeech(data, model='gtts', output_folder='output'):
    """
    Fonction permettant de traiter les données du dataset LibriSpeech
    :param data: Données à traiter
    :param model: Modèle à utiliser (Default gtts)
    :param output_folder: Dossier de sortie (Default output)
    :return: ID, file, checksum, transcript, (audio, sample_rate)
    """
    (waveform, sample_rate, transcript, ID_s, ID_c, ID_u) = data
    
    waveform = waveform.squeeze(0)
    sample_rate = sample_rate.squeeze(0)
    transcript = str(transcript[0])
    ID = "-".join((str(ID_s.item()), str(ID_c.item()), str(ID_u.item()))).replace(',','')
    
    file = f'./{output_folder}/out_{ID}.mp3'
    
    if os.path.exists(file):
        return ID, file, 1, None, None
    
    return ID, file, 0, transcript, (waveform, sample_rate)


def load_GEC_dataset(path='data\\GEC', text=None, dataset=GEC_Dataset):
    """
    Fonction permettant de charger le dataset GEC
    :param path: Chemin du dataset
    :param text: Texte corrigé (Default None, on utilise les données du path. Pour une utilisation du TTS en pipeline alors text est le texte corrigé)
    :param dataset: Classe du dataset à utiliser (Default GEC_Dataset)
    :return: Dataloader du dataset
    """
    GEC_Dataset = dataset(path, text)
    GEC_Dataloader = DataLoader(GEC_Dataset, batch_size=1)
    
    print("Number of samples:", len(GEC_Dataset))
    
    sample_meta0 = GEC_Dataloader.dataset[0]
    print("""
    Transcript n°0 :
    
    Transcript : {}
    Transcript ID : {}
    """.format(*sample_meta0))
    
    return GEC_Dataloader


def process_GEC(data, model='gtts', output_folder='output'):
    """
    Fonction permettant de traiter les données du dataset GEC
    :param data: Données à traiter
    :param model: Modèle à utiliser (Default gtts)
    :param output_folder: Dossier de sortie (Default output)
    :return: ID, file, checksum, transcript, (audio, sample_rate)
    """
    
    (transcript, ID) = data
    ID = ID.item()
    file = f'./{output_folder}/out_{ID}.mp3'
    
    return ID, file, 0, transcript, None


___
## Création de la fonction TTS

Nous allons ici définir la fonction TTS qui va nous permettre de générer des fichiers audios à partir d'un texte. Cette fonction prend en entrée un texte et renvoie un fichier audio (waveform, sample_rate). En mode test, cette fonction prend en entrée le chemin d'un dataset.

In [4]:
def TTS(text:str=None, model='gtts', output_folder='output', dataset='GEC'):
    """
    Fonction permettant de générer des fichiers audios à partir d'un texte
    :param text: Texte à transformer en audio (Default None, on utilise les données du dataset. Pour une utilisation du TTS en pipeline alors text est le texte corrigé)
    :param model: Modèle à utiliser (Default gtts)
    :param output_folder: Dossier de sortie (Default output)
    :param dataset: Dataset à utiliser (Default GEC)
    :return: Fichier audio généré (waveform, sample_rate)
    """
    # Création du dossier de sortie
    pipeline = "/pipeline/" if text is not None else "/"
    output_folder = output_folder + pipeline + model + '_' + dataset
    
    os.makedirs(output_folder, exist_ok=True)
    
    # TQDM loader et fonction de traitement des données
    try:
        function = "load_" + dataset + "_dataset"
        dataloader = eval(function)(text=text)
        dataloader_tqdm = tqdm(dataloader, total=len(dataloader))
    
        process_function = "process_" + dataset
        
    except NameError:
        raise ValueError(f'Unknown dataset {dataset}')
    
    # Création du modèle
    bundle = torchaudio.pipelines.TACOTRON2_WAVERNN_PHONE_LJSPEECH
    processor = bundle.get_text_processor()
    tacotron2 = bundle.get_tacotron2().to("cuda")
    file_out=''
    print(f'Generating audio files with {model} model in {output_folder} folder')
    
    # Boucle sur les données
    for i, data in enumerate(dataloader_tqdm):
        # Traitement des données
        (ID, file_out, out, transcript, evalua) = eval(process_function)(data, model=model, output_folder=output_folder)
        file_out = file_out if type(file_out) == str else file_out[0]
        transcript = str(transcript)
        
        # Vérification du checksum
        if out == 1:
            dataloader_tqdm.set_postfix({'status': 'Skipped', 'ID': ID})
            continue
        
        # Synthèse du texte
        try:
            #Passage du transcript dans le modèle
            if model == 'gtts':
                tts = gTTS(transcript, lang='en')#, tld='co.in')
                tts.save(file_out)
            elif model == 'tacotron2':
                with torch.inference_mode():
                    processed, lengths = processor(transcript)
                    processed = processed.to("cuda")
                    lengths = lengths.to("cuda")
                    spec, spec_l, _ = tacotron2.infer(processed, lengths)

                plt.imshow(spec[0].cpu().detach(), origin="lower", aspect="auto", cmap="viridis")
                plt.show()

            else:
                raise ValueError(f'Unknown model {model}')
            
            dataloader_tqdm.set_postfix()
        

        except Exception as e:
            dataloader_tqdm.set_postfix({'status': 'Error', 'ID': ID})
            raise e
            
        # Evaluation
        if evalua != None:
            waveform, sample_rate = evalua
            transform = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate, n_fft=1024, win_length=1024, hop_length=256)
            mel_specgram = transform(waveform)
            mag_spect = torch.stft(waveform, 1024, window=torch.hamming_window(1024), return_complex=True, hop_length=256)
            mag_spect = torch.sqrt(torch.abs(mag_spect.permute(1,2,0)))
            mel_specgram = torch.sqrt(torch.abs(mel_specgram))
            plt.imshow(mel_specgram[0], origin="lower", aspect="auto", cmap="viridis")
            from sklearn.metrics.pairwise import cosine_similarity
            import numpy as np

            spec1 = mel_specgram[0].cpu().detach()
            spec2 = spec.squeeze().cpu().detach()
            
            
            shape1 = spec1.shape
            shape2 = spec2.shape
            
            size_diff_rows = shape2[0] - shape1[0]
            size_diff_cols = shape2[1] - shape1[1]
            
            if size_diff_rows > 0:      
                padded_spec1 = np.pad(spec1, ((0, size_diff_rows), (0, 0)), mode='constant')
            else:
                padded_spec1 = spec1
            
            if size_diff_cols > 0:
                padded_spec2 = np.pad(spec2, ((0, 0), (0, size_diff_cols)), mode='constant')
            else:
                padded_spec2 = spec2


            flat_spec1 = padded_spec1.flatten()
            flat_spec2 = padded_spec2.flatten()
            min_size = min(len(flat_spec1), len(flat_spec2))
            flat_spec1 = flat_spec1[:min_size]
            flat_spec2 = flat_spec2[:min_size]
            
            # Compute cosine similarity
            similarity_measure = cosine_similarity([flat_spec1], [flat_spec2])[0, 0]
            print(similarity_measure)
            
            mse = np.mean((padded_spec1 - padded_spec2)**2)
            similarity_measure = 1 / (1 + mse) 
            print(similarity_measure)
            
            from fastdtw import fastdtw

            # Assuming spec1 and spec2 are the normalized spectrograms
            distance, path = fastdtw(padded_spec1, padded_spec2)
            similarity_measure = 1 / (1 + distance)  # Invert to get a similarity measure
            print(similarity_measure)
    
    
    return torchaudio.load(file_out) if file_out != '' else None

___
## TTS des modèles et des datasets

In [5]:
# # gTTS avec dataset GEC (pipeline)
audio, rate = TTS("I dont have a car but i dreaming it off")
ipd.Audio(audio, rate=rate)

Number of samples: 1

    Transcript n°0 :
    
    Transcript : I dont have a car but i dreaming it off
    Transcript ID : 0
    


  0%|          | 0/1 [00:00<?, ?it/s]

D:\Projets\env\3.10\utils_env\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Generating audio files with gtts model in output/pipeline/gtts_GEC folder


In [6]:
# # Tacotron2 avec dataset LibriSpeech
# TTS(model='tacotron2', output_folder='output', dataset='librispeech')

In [7]:
# # gTTS avec dataset GEC
# TTS(model="gtts", output_folder="output", dataset="GEC")